# Procurement SQL Analysis

This notebook analyzes procurement and supplier performance data using SQL.
The analysis focuses on spend, compliance, negotiation savings, and quality metrics
using a real procurement KPI dataset.

In [2]:
import pandas as pd
import sqlite3

In [3]:
df = pd.read_csv("Procurement KPI Analysis Dataset.csv")
df.head()

,PO_ID,Supplier,Order_Date,Delivery_Date,Item_Category,Order_Status,Quantity,Unit_Price,Negotiated_Price,Defective_Units,Compliance
0,PO-00001,Alpha_Inc,10/17/2023,10/25/2023,Office Supplies,Cancelled,1176,20.13,17.81,NaN,Yes
1,PO-00002,Delta_Logistics,4/25/2022,5/5/2022,Office Supplies,Delivered,1509,39.32,37.34,235.0,Yes
2,PO-00003,Gamma_Co,1/26/2022,2/15/2022,MRO,Delivered,910,95.51,92.26,41.0,Yes
3,PO-00004,Beta_Supplies,10/9/2022,10/28/2022,Packaging,Delivered,1344,99.85,95.52,112.0,Yes
4,PO-00005,Delta_Logistics,9/8/2022,9/20/2022,Raw Materials,Delivered,1180,64.07,60.53,171.0,No


In [4]:
df.columns = (
    df.columns
    .str.strip()
    .str.lower()
    .str.replace(" ", "_")
)

df.columns

Index(['po_id', 'supplier', 'order_date', 'delivery_date', 'item_category',
       'order_status', 'quantity', 'unit_price', 'negotiated_price',
       'defective_units', 'compliance'],
      dtype='object')

In [5]:
conn = sqlite3.connect("procurement.db")

df.to_sql(
    name="procurement_orders",
    con=conn,
    if_exists="replace",
    index=False
)

pd.read_sql_query(
    "SELECT COUNT(*) AS rows_loaded FROM procurement_orders;",
    conn
)

,rows_loaded
0,777


In [6]:
pd.read_sql_query("""
SELECT supplier,
       SUM(quantity * negotiated_price) AS total_spend
FROM procurement_orders
GROUP BY supplier
ORDER BY total_spend DESC;
""", conn)

,supplier,total_spend
0,Beta_Supplies,9858665.90
1,Epsilon_Group,9851156.06
2,Delta_Logistics,9236240.47
3,Gamma_Co,8587921.71
4,Alpha_Inc,7839712.25


In [7]:
pd.read_sql_query("""
SELECT compliance,
       COUNT(*) AS order_count
FROM procurement_orders
GROUP BY compliance;
""", conn)

,compliance,order_count
0,No,137
1,Yes,640


In [8]:
pd.read_sql_query("""
SELECT supplier,
       AVG(unit_price - negotiated_price) AS avg_savings_per_unit
FROM procurement_orders
GROUP BY supplier
ORDER BY avg_savings_per_unit DESC;
""", conn)

,supplier,avg_savings_per_unit
0,Epsilon_Group,4.757229
1,Alpha_Inc,4.738369
2,Beta_Supplies,4.671538
3,Gamma_Co,4.642168
4,Delta_Logistics,4.337719


In [9]:
df_final = pd.read_sql_query("""
SELECT supplier,
       SUM(quantity * negotiated_price) AS total_spend
FROM procurement_orders
GROUP BY supplier
ORDER BY total_spend DESC;
""", conn)

df_final.to_csv("kaggle_procurement_supplier_spend.csv", index=False)

In [10]:
conn.close()